# Experiment 005: Zaburo's Row-Based Approach

Implement Zaburo's row-based grid placement which is GUARANTEED to have no overlaps:
- Trees arranged in rows with alternating angles (0° and 180°)
- Horizontal spacing: 0.7 (tree width)
- Vertical spacing: 1.0 (tree height)
- Expected score: ~88.33

This is the ONLY safe path forward since all pre-optimized baselines have overlaps.

In [1]:
import math
import os
from decimal import Decimal, getcontext
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json
from tqdm import tqdm

# Use high precision
getcontext().prec = 25
scale_factor = Decimal('1e15')

print(f"Decimal precision: {getcontext().prec}")
print(f"Scale factor: {scale_factor}")

Decimal precision: 25
Scale factor: 1E+15


In [2]:
# ChristmasTree class - EXACT copy from getting-started kernel
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self.polygon = self._create_polygon()
    
    def _create_polygon(self):
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        return affinity.translate(rotated,
                                  xoff=float(self.center_x * scale_factor),
                                  yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def check_overlap(trees):
    """Check for overlaps - must match Kaggle's validation."""
    polygons = [t.polygon for t in trees]
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return True  # HAS OVERLAP
    return False  # No overlap

def calculate_side_length(trees):
    """Calculate bounding box side length."""
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(str(bounds[0])) / scale_factor
    miny = Decimal(str(bounds[1])) / scale_factor
    maxx = Decimal(str(bounds[2])) / scale_factor
    maxy = Decimal(str(bounds[3])) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return max(width, height)

print("Helper functions defined")

Helper functions defined


In [4]:
def find_best_trees_zaburo(n):
    """
    Zaburo's row-based placement - GUARANTEED no overlaps.
    
    Trees are arranged in rows:
    - Even rows (0, 2, 4, ...): angle=0, y = row_idx * 1.0
    - Odd rows (1, 3, 5, ...): angle=180, y = 0.8 + (row_idx-1) * 1.0, x_offset = 0.35
    - Horizontal spacing: 0.7
    
    We try different row configurations to find the best one.
    """
    best_score = float('inf')
    best_trees = None
    
    # Try different numbers of trees per row
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1, n_even + 1]:
            if n_odd < 0:
                continue
            
            all_trees = []
            rest = n
            r = 0  # row index
            
            while rest > 0:
                # Determine how many trees in this row
                if r % 2 == 0:
                    m = min(rest, n_even)
                else:
                    m = min(rest, n_odd)
                
                if m <= 0:
                    break
                    
                rest -= m
                
                # Row parameters
                angle = 0 if r % 2 == 0 else 180
                x_offset = Decimal('0') if r % 2 == 0 else Decimal('0.35')  # 0.7/2
                
                if r % 2 == 0:
                    y = Decimal(str(r // 2)) * Decimal('1.0')
                else:
                    y = Decimal('0.8') + Decimal(str((r - 1) // 2)) * Decimal('1.0')
                
                # Create trees for this row
                for i in range(m):
                    x = Decimal('0.7') * Decimal(str(i)) + x_offset
                    tree = ChristmasTree(center_x=str(x), center_y=str(y), angle=str(angle))
                    all_trees.append(tree)
                
                r += 1
            
            if len(all_trees) != n:
                continue
            
            # Calculate score
            side = calculate_side_length(all_trees)
            score = float(side ** 2) / n
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print("Zaburo's row-based function defined")

Zaburo's row-based function defined


In [5]:
# Special case for N=1: optimal is 45 degrees at origin
def get_optimal_n1():
    """N=1 is optimal at 45 degrees."""
    tree = ChristmasTree(center_x='0', center_y='0', angle='45')
    side = calculate_side_length([tree])
    score = float(side ** 2) / 1
    return score, [tree]

print("N=1 optimal function defined")

N=1 optimal function defined


In [ ]:
# Generate all configurations
all_trees = {}
per_n_scores = {}

print("Generating configurations for N=1 to N=200...")
for n in tqdm(range(1, 201)):
    if n == 1:
        score, trees = get_optimal_n1()
    else:
        score, trees = find_best_trees_zaburo(n)
    
    all_trees[n] = trees
    per_n_scores[n] = score

print(f"\nGenerated all configurations")

In [ ]:
# Validate ALL N values
print("Validating all N values...")
overlapping_n = []

for n in tqdm(range(1, 201), desc="Validating"):
    if check_overlap(all_trees[n]):
        overlapping_n.append(n)

print(f"\nTotal N values with overlaps: {len(overlapping_n)}")
if overlapping_n:
    print(f"Overlapping N values: {overlapping_n}")
else:
    print("✓ All N values validated - NO OVERLAPS!")

In [ ]:
# Calculate total score
total_score = sum(per_n_scores.values())

print(f"Total score: {total_score:.6f}")
print(f"\nComparison:")
print(f"  Expected (Zaburo): ~88.33")
print(f"  Our implementation: {total_score:.6f}")
print(f"  Greedy (exp_003):   169.46")
print(f"  Target:             68.89")

print(f"\nTop 10 score contributors:")
sorted_scores = sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:10]:
    side = calculate_side_length(all_trees[n])
    print(f"N={n:3d}: side={float(side):.6f}, score={score:.6f}")

In [ ]:
# Create submission DataFrame
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]
tree_data = []

for n in range(1, 201):
    for tree in all_trees[n]:
        tree_data.append([tree.center_x, tree.center_y, tree.angle])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

# Format EXACTLY as in getting-started kernel
for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)
    
# Prepend 's' to ensure string format
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(f"Submission shape: {submission.shape}")
print(f"\nFirst 10 rows:")
print(submission.head(10))

In [ ]:
# Save submission
os.makedirs('/home/submission', exist_ok=True)
submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/005_zaburo_rowbased/submission.csv')

# Save metrics
metrics = {
    'cv_score': total_score,
    'overlapping_n_count': len(overlapping_n),
    'per_n_scores': {str(k): v for k, v in per_n_scores.items()}
}

with open('/home/code/experiments/005_zaburo_rowbased/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nSubmission saved!")
print(f"CV Score: {total_score:.6f}")
print(f"Overlapping N values: {len(overlapping_n)}")
print(f"\nThis is Zaburo's row-based approach - GUARANTEED no overlaps by construction.")
print(f"Should be ACCEPTED by Kaggle.")